Copyright (c) 2020-2021. All rights reserved.

Licensed under the MIT License.

# HPO for Fine-Tuning Pre-trained Language Models

*Running this notebook takes about one hour.

## 1. Introduction


In this notebook, we demonstrate a procedure for troubleshooting HPO failure in fine-tuning pre-trained language models (introduced in the following paper):

*An Empirical Study on Hyperparameter Optimization for Fine-Tuning Pre-trained Language Models. Xueqing Liu, Chi Wang. To appear in ACL-IJCNLP 2021*

FLAML requires `Python>=3.6`. To run this notebook example, please install flaml with the `notebook` and `nlp` options:
```bash
pip install flaml[nlp]
```
Our paper was developed under transformers version 3.4.0. We uninstall and reinstall transformers==3.4.0:

In [ ]:
!pip install flaml[nlp]
!pip install transformers==3.4.0
from flaml.nlp import AutoTransformers


## 2. Initial Experimental Study (Section 4)


### Load dataset 

Load the dataset using AutoTransformer.prepare_data. In this notebook, we use the Microsoft Research Paraphrasing Corpus (MRPC) dataset and the Electra model as an example:

In [2]:
autohf = AutoTransformers()
preparedata_setting = {
        "dataset_subdataset_name": "glue:mrpc",
        "pretrained_model_size": "google/electra-base-discriminator:base",
        "data_root_path": "data/",
        "max_seq_length": 128,
        }
autohf.prepare_data(**preparedata_setting)


console_args has no attribute pretrained_model_size, continue
console_args has no attribute dataset_subdataset_name, continue
console_args has no attribute algo_mode, continue
console_args has no attribute space_mode, continue
console_args has no attribute search_alg_args_mode, continue
console_args has no attribute algo_name, continue
console_args has no attribute pruner, continue
console_args has no attribute resplit_mode, continue
console_args has no attribute rep_id, continue
console_args has no attribute seed_data, continue
console_args has no attribute seed_transformers, continue
console_args has no attribute learning_rate, continue
console_args has no attribute weight_decay, continue


Reusing dataset glue (/home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-6a78e5c95406457c.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-e8d0f3e04c3b4588.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-4b0966b394994163.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-52e85ab7216b1e98.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c996572

### Running grid search

First, we run grid search using Electra. By specifying `algo_mode="grid"`, AutoTransformers will run the grid search algorithm. By specifying `space_mode="grid"`, AutoTransformers will use the default grid search configuration recommended by the Electra paper:

In [3]:
import transformers
autohf_settings = {"resources_per_trial": {"gpu": 1, "cpu": 1},
                   "num_samples": 1,
                   "time_budget": 100000,  # unlimited time budget
                   "ckpt_per_epoch": 5,
                   "fp16": True,
                   "algo_mode": "grid",  # set the search algorithm to grid search
                   "space_mode": "grid", # set the search space to the recommended grid space
                   "transformers_verbose": transformers.logging.ERROR
                   }
validation_metric, analysis = autohf.fit(**autohf_settings)

2021-06-16 10:04:33,114	INFO tune.py:450 -- Total run time: 149.77 seconds (149.63 seconds for the tuning loop).


Total running time: 149.798171043396 seconds


Get the time for running grid search: 

In [4]:
GST = autohf.last_run_duration
print("grid search for {} took {} seconds".format(autohf.jobid_config.get_jobid_full_data_name(), GST))

grid search for glue_mrpc took 149.798171043396 seconds


After the HPO run finishes, generate the predictions and save it as a .zip file to be submitted to the glue website. Here we will need the library AzureUtils which is for storing the output information (e.g., analysis log, .zip file) locally and uploading the output to an azure blob container (e.g., if multiple jobs are executed in a cluster). If the azure key and container information is not specified, the output information will only be saved locally. 

In [5]:
predictions, test_metric = autohf.predict()
from flaml.nlp.result_analysis.azure_utils import AzureUtils

print(autohf.jobid_config)

azure_utils = AzureUtils(root_log_path="logs_test/", autohf=autohf)
azure_utils.write_autohf_output(valid_metric=validation_metric,
                                predictions=predictions,
                                duration=GST)
print(validation_metric)

remove_columns_ is deprecated and will be removed in the next major version of datasets. Use the dataset.remove_columns method instead.


Cleaning the existing label column from test data


JobID(dat=['glue'], subdat='mrpc', mod='grid', spa='grid', arg='dft', alg='grid', pru='None', pre_full='google/electra-base-discriminator', pre='electra', presz='base', spt='ori', rep=0, sddt=43, sdhf=42, var1=None, var2=None)
Your output will not be synced to azure because azure key and container name are not specified
The path for saving the prediction .zip file is not specified, setting to data/ by default
Your output will not be synced to azure because azure key and container name are not specified
{'eval_accuracy': 0.7254901960784313, 'eval_f1': 0.8276923076923076, 'eval_loss': 0.516851007938385}


The validation F1/accuracy we got was 92.4/89.5. After the above steps, you will find a .zip file for the predictions under data/result/. Submit the .zip file to the glue website. The test F1/accuracy we got was 90.4/86.7. As an example, we only run the experiment one time, but in general, we should run the experiment multiple repetitions and report the averaged validation and test accuracy.

### Running Random Search

Next, we run random search with the same time budget as grid search:

In [6]:
def tune_hpo(time_budget, this_hpo_space):
    autohf_settings = {"resources_per_trial": {"gpu": 1, "cpu": 1},
                       "num_samples": -1,
                       "time_budget": time_budget,
                       "ckpt_per_epoch": 5,
                       "fp16": True,
                       "algo_mode": "hpo",  # set the search algorithm mode to hpo
                       "algo_name": "rs",
                       "space_mode": "cus", # customized search space (this_hpo_space)
                       "hpo_space": this_hpo_space,
                       "transformers_verbose": transformers.logging.ERROR
                       }
    validation_metric, analysis = autohf.fit(**autohf_settings)
    predictions, test_metric = autohf.predict()
    from flaml.nlp.result_analysis.azure_utils import AzureUtils
    azure_utils = AzureUtils(root_log_path="logs_test/", autohf=autohf)
    azure_utils.write_autohf_output(valid_metric=validation_metric,
                                    predictions=predictions,
                                    duration= GST)
    print(validation_metric)

In [7]:
hpo_space_full = {
               "learning_rate": {"l": 3e-5, "u": 1.5e-4, "space": "log"},
               "warmup_ratio": {"l": 0, "u": 0.2, "space": "linear"},
               "num_train_epochs": [0.5],
               "per_device_train_batch_size": [16, 32, 64],
               "weight_decay": {"l": 0.0, "u": 0.3, "space": "linear"},
               "attention_probs_dropout_prob": {"l": 0, "u": 0.2, "space": "linear"},
               "hidden_dropout_prob": {"l": 0, "u": 0.2, "space": "linear"},
            }

tune_hpo(GST, hpo_space_full)

(pid=19568) {'eval_loss': 0.5411379933357239, 'eval_accuracy': 0.7303921568627451, 'eval_f1': 0.8318042813455658, 'epoch': 0.5}
(pid=19568) {'eval_loss': 0.5411379933357239, 'eval_accuracy': 0.7303921568627451, 'eval_f1': 0.8318042813455658, 'epoch': 0.5}
(pid=19563) {'eval_loss': 0.5757343173027039, 'eval_accuracy': 0.7083333333333334, 'eval_f1': 0.8226527570789866, 'epoch': 0.5}
(pid=19563) {'eval_loss': 0.5757343173027039, 'eval_accuracy': 0.7083333333333334, 'eval_f1': 0.8226527570789866, 'epoch': 0.5}
(pid=19567) {'eval_loss': 0.628964900970459, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5}
(pid=19567) {'eval_loss': 0.628964900970459, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5}


2021-06-16 10:08:06,998	INFO tune.py:450 -- Total run time: 161.60 seconds (150.69 seconds for the tuning loop).


Total running time: 161.6244192123413 seconds


Your output will not be synced to azure because azure key and container name are not specified
The path for saving the prediction .zip file is not specified, setting to data/ by default
Your output will not be synced to azure because azure key and container name are not specified
{'eval_accuracy': 0.7328431372549019, 'eval_f1': 0.8320493066255777, 'eval_loss': 0.5431155562400818}


The validation F1/accuracy we got was 93.5/90.9. Similarly, we can submit the .zip file to the glue website. The test F1/accuaracy we got was 81.6/70.2. 

## 3. Troubleshooting HPO Failures

Since the validation accuracy is larger than grid search while the test accuracy is smaller, HPO has overfitting. We reduce the search space:

In [8]:
hpo_space_fixwr = {
               "learning_rate": {"l": 3e-5, "u": 1.5e-4, "space": "log"},
               "warmup_ratio": [0.1],
               "num_train_epochs": [0.5],
               "per_device_train_batch_size": [16, 32, 64],
               "weight_decay": {"l": 0.0, "u": 0.3, "space": "linear"},
               "attention_probs_dropout_prob": {"l": 0, "u": 0.2, "space": "linear"},
               "hidden_dropout_prob": {"l": 0, "u": 0.2, "space": "linear"},
            }
tune_hpo(GST, hpo_space_fixwr)

(pid=22977) {'eval_loss': 0.6253387928009033, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5043478260869565}
(pid=22977) {'eval_loss': 0.6253387928009033, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5043478260869565}
(pid=22977) {'eval_loss': 0.6253387928009033, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5043478260869565}
(pid=22981) {'eval_loss': 0.6236189007759094, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5043478260869565}
(pid=22981) {'eval_loss': 0.6236189007759094, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5043478260869565}
(pid=22981) {'eval_loss': 0.6236189007759094, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.5043478260869565}


2021-06-16 10:11:27,073	INFO tune.py:450 -- Total run time: 157.41 seconds (151.72 seconds for the tuning loop).


Total running time: 157.44810509681702 seconds


Your output will not be synced to azure because azure key and container name are not specified
The path for saving the prediction .zip file is not specified, setting to data/ by default
Your output will not be synced to azure because azure key and container name are not specified
{'eval_accuracy': 0.7279411764705882, 'eval_f1': 0.8268330733229329, 'eval_loss': 0.5298551321029663}


The validation F1/accuracy we got was 92.6/89.7, the test F1/accuracy was 85.9/78.7, therefore overfitting still exists and we further reduce the space: 

In [9]:
hpo_space_min = {
               "learning_rate": {"l": 3e-5, "u": 1.5e-4, "space": "log"},
               "warmup_ratio": [0.1],
               "num_train_epochs": [0.5],
               "per_device_train_batch_size": [16, 32, 64],
               "weight_decay": [0.0],
               "attention_probs_dropout_prob": [0.1],
               "hidden_dropout_prob": [0.1],
            }
tune_hpo(GST, hpo_space_min)

(pid=26361) {'eval_loss': 0.6146416664123535, 'eval_accuracy': 0.7107843137254902, 'eval_f1': 0.8233532934131738, 'epoch': 0.41739130434782606}
(pid=26361) {'eval_loss': 0.6146416664123535, 'eval_accuracy': 0.7107843137254902, 'eval_f1': 0.8233532934131738, 'epoch': 0.41739130434782606}
(pid=26361) {'eval_loss': 0.6146416664123535, 'eval_accuracy': 0.7107843137254902, 'eval_f1': 0.8233532934131738, 'epoch': 0.41739130434782606}
(pid=26361) {'eval_loss': 0.6146416664123535, 'eval_accuracy': 0.7107843137254902, 'eval_f1': 0.8233532934131738, 'epoch': 0.41739130434782606}
(pid=26414) {'eval_loss': 0.6239543557167053, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.4}
(pid=26414) {'eval_loss': 0.6239543557167053, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.4}
(pid=26414) {'eval_loss': 0.6239543557167053, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'epoch': 0.4}
(pid=26414) {'eval_loss': 0.623954355716

2021-06-16 10:15:00,567	INFO tune.py:450 -- Total run time: 171.05 seconds (150.43 seconds for the tuning loop).


Total running time: 171.07985877990723 seconds


Your output will not be synced to azure because azure key and container name are not specified
The path for saving the prediction .zip file is not specified, setting to data/ by default
Your output will not be synced to azure because azure key and container name are not specified
{'eval_accuracy': 0.7181372549019608, 'eval_f1': 0.8174962292609351, 'eval_loss': 0.5494586229324341}


The validation F1/accuracy we got was 90.4/86.7, test F1/accuracy was 83.0/73.0. Since the validation accuracy is below grid search, we increase the budget to 4 * GST:

In [ ]:
hpo_space_min = {
               "learning_rate": {"l": 3e-5, "u": 1.5e-4, "space": "log"},
               "warmup_ratio": [0.1],
               "num_train_epochs": [0.5],
               "per_device_train_batch_size": [16, 32, 64],
               "weight_decay": [0.0],
               "attention_probs_dropout_prob": [0.1],
               "hidden_dropout_prob": [0.1],
            }
tune_hpo(4 * GST, hpo_space_min)

(pid=29786) {'eval_loss': 0.5486209392547607, 'eval_accuracy': 0.7034313725490197, 'eval_f1': 0.8141321044546851, 'epoch': 0.5}
(pid=29786) {'eval_loss': 0.5486209392547607, 'eval_accuracy': 0.7034313725490197, 'eval_f1': 0.8141321044546851, 'epoch': 0.5}
(pid=29786) {'eval_loss': 0.5486209392547607, 'eval_accuracy': 0.7034313725490197, 'eval_f1': 0.8141321044546851, 'epoch': 0.5}
(pid=29786) {'eval_loss': 0.5486209392547607, 'eval_accuracy': 0.7034313725490197, 'eval_f1': 0.8141321044546851, 'epoch': 0.5}
(pid=29786) {'eval_loss': 0.5486209392547607, 'eval_accuracy': 0.7034313725490197, 'eval_f1': 0.8141321044546851, 'epoch': 0.5}


The validation F1/accuracy we got was 92.3/89.7, where the accuracy outperforms grid search. The test F1/accuracy was 91.1/87.8. As a result, random search with 4*GST and the minimal search space `hpo_space_min` has outperformed grid search. We stop the troubleshooting process. 